# Demo Notebook how to run models on static mouse datasets

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True

name = "test"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [3]:
import torch
import numpy as np
import pickle 
import pandas as pd
from collections import OrderedDict, Iterable

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder

from nnsysident.tables.experiments import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

<ipython-input-3-71e7f2af38a6>:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import OrderedDict, Iterable


Connecting konstantin@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306


# Get Dataloader

In [ ]:
# change path here
paths = ['data/static22564-2-12-preproc0.zip',
                     'data/static22564-2-13-preproc0.zip',
                     'data/static22564-3-8-preproc0.zip',
                     'data/static22564-3-12-preproc0.zip']

#paths = ['data/static22564-2-12-preproc0.zip']
dataset_fn = 'nnsysident.datasets.mouse_loaders.static_shared_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    seed=1,
    image_n=50,
    image_base_seed=1,
    multi_match_n=972,
    multi_match_base_seed=1,
    exclude_multi_match_n = 3625,

)
dataloaders = builder.get_data(dataset_fn, dataset_config)

# Get Model

### Spatial Transformer

In [ ]:
# model_fn = 'nnsysident.models.models.se2d_spatialxfeaturelinear'
# model_config = {
#    'pad_input': False,
#    'stack': -1,
#    'layers':4,
#    'input_kern': 9,
#    'gamma_input': 20,
#    'gamma_readout': 0.005,
#    'hidden_dilation': 1,
#    'hidden_kern': 7,
#    'hidden_channels': 64,
#     'init_noise': 1e-3,
#     'depth_separable': True,
    
# }
# model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

### The New gaussian readout: change gauss_type for the different modes

In [ ]:
model_fn = 'nnsysident.models.models.taskdriven_fullgaussian2d'

model_config = {'tl_model_name': 'vgg16',
   'layers': 8,
   'init_mu_range': 0.55,
   'init_sigma': 0.4,
   'share_features': False,
   'grid_mean_predictor': {'type': 'cortex',
    'input_dimensions': 2,
    'hidden_layers': 0,
    'hidden_features': 0,
    'final_tanh': False},
   'gamma_readout': 4.622488854650272}


model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)

In [ ]:
model

In [ ]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'

model_config = {'share_features': True,
 'init_mu_range': 0.55,
 'init_sigma': 0.4,
 'input_kern': 15,
 'hidden_kern': 13,
 'gamma_input': 1.0,
 'gamma_readout': 2.117604964706911,
 'grid_mean_predictor': {'type': 'cortex',
  'input_dimensions': 2,
  'hidden_layers': 0,
  'hidden_features': 0,
  'final_tanh': False}}


model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)

In [ ]:
torch.mm(model.readout['22564-3-12-0'].mu_transform[0].weight[:, 0].view(1, -1),
         model.readout['22564-3-12-0'].mu_transform[0].weight[:, 1].view(1, -1).T).abs()

In [ ]:
core_dict = OrderedDict([(k, v) for k, v in torch.load('inshallahmodel.mojo').items() if k[0:5] == 'core.'])
model.load_state_dict(core_dict, strict=False)

In [ ]:
model.readout.regularizer('22564-2-12-0')

In [ ]:
model.readout.regularizer('22564-2-12-0')

In [ ]:
model.readout.gamma_readout

# Get Trainer

In [ ]:
trainer_fn = 'nnsysident.training.trainers.standard_trainer'
trainer_config = dict(detach_core=True, track_training=True)
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [ ]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1)

In [ ]:
# grid = model.readout['22564-2-12-0'].mu.squeeze().cpu().data.numpy()
for data_key in ['22564-3-8-0', '22564-3-12-0', '22564-2-13-0', '22564-2-12-0']:
   # model.readout[data_key]

    
    grid = model.readout[data_key].mu.squeeze().detach().cpu().numpy()
    plt.scatter(*grid.T)
    plt.xlim([-1,1])
    plt.ylim([-1,1])
    plt.show()

In [ ]:
plt.scatter(*model.readout['22564-3-12-0'].source_grid.T.cpu().numpy())

In [ ]:
# grid = model.readout['22564-2-12-0'].mu.squeeze().cpu().data.numpy()
grid = model.readout['22564-3-12-0'].mu.squeeze().detach().cpu().numpy()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(*grid.T)

In [ ]:
plt.scatter(*dataloaders['train']['22564-2-12-0'].dataset.neurons.[:,:2].T)

In [ ]:
torch.mm(model.readout['22564-3-12-0'].mu_transform[0].weight[:, 0].view(1, -1),
         model.readout['22564-3-12-0'].mu_transform[0].weight[:, 1].view(1, -1).T).abs()

In [ ]:
model.readout['22564-3-12-0'].mu_transform[0].weight

In [ ]:
model.readout['22564-3-12-0'].mu_transform[0].weight

In [ ]:
torch.norm(model.readout['22564-3-12-0'].mu_transform[0].weight, dim=0, p=2)

In [ ]:
torch.abs(torch.norm(model.readout['22564-3-12-0'].mu_transform[0].weight[:, 0], p=2) - 
          torch.norm(model.readout['22564-3-12-0'].mu_transform[0].weight[:, 1], p=2))

In [ ]:
aspect = agg_fn(torch.abs(torch.norm(self.trans_mat[:, :, 0]**2, dim=1) -
                          torch.norm(self.trans_mat[:, :, 1]**2, dim=1)))

In [ ]:
l.weight

In [ ]:
model.core.features.layer0.conv.weight.device

In [ ]:
model.readout['22564-3-12-0'].source_grid.shape

In [ ]:
sg = model.readout['22564-3-12-0'].source_grid.cpu().data.numpy()

In [ ]:
plt.scatter(*sg.T)

In [ ]:
x, y = sg.max(axis=0) - sg.min(axis=0)
x/y

In [ ]:
sg